In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd

from hist import Hist

import babar_analysis_tools as bat
import myPIDselector

import os 

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

# At home (Bellis)
topdir = '/home/bellis/babar_data/bnv_plambda'


### Read in SP- includes both signal and background 
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

### convert files from a parquet file to an awkward array
start = time.time()
ak_sp = ak.from_parquet(filename)
print(f"Took {time.time() - start} s")


# Read in data
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#start = time.time()
#ak_data = ak.from_parquet(filename)
#print(f"Took {time.time() - start} s")


In [ ]:
#spmask = ak_sp['spmode']=='-999'
#spmask = ak_sp['spmode']=='998'
#spmask = ak_sp['spmode']=='1005'


#ak_sp = ak_sp[spmask]

In [ ]:
q = (ak_sp['Bd2Lund'][:,0])/np.abs(ak_sp['Bd2Lund'][:,0])
q

In [ ]:
eps = myPIDselector.PIDselector("e")
pps = myPIDselector.PIDselector("p")
pips = myPIDselector.PIDselector("pi")
Kps = myPIDselector.PIDselector("K")
mups = myPIDselector.PIDselector("mu")

In [ ]:
lamd1idx = ak_sp['Lambda0d1Idx']
lamd1Lund = ak_sp['Lambda0d1Lund']

print(f'lamd1idx\n{lamd1idx}')
print(f'lamd1Lund\n{lamd1Lund}')

d2idx = ak_sp['Bd2Idx']
d2Lund = ak_sp['Bd2Lund']

print(f'd2idx\n{d2idx}')
print(f'd2Lund\n{d2Lund}')

trkidx_proton = ak_sp['pTrkIdx']
trk_selector_map = ak_sp['pSelectorsMap']
print()

print("trk_selector_map")
print(trk_selector_map)

# Do this for all the tracks
pbits = bat.calculate_bits_for_PID_selector(trkidx_proton, trk_selector_map, verbose=1)
print(f'pbits:\n{pbits}')
print(f'pbits[d2idx]:\n{pbits[d2idx]}')
print()

selector = 'LooseKMProtonSelection'
print(f"Now trying to create a mask with {selector}")

mask_bool = bat.mask_PID_selection(pbits[d2idx], selector, pps)
print(f'mask_book: {mask_bool}')
print()
passing_pbits = pbits[d2idx][mask_bool]
print("\nMasked bits of the daughters...")
print(f'{passing_pbits}')
print()

print("\nMasked bits of the not daughters...")
passing_pbits_not_daughter = pbits[~d2idx][mask_bool]
print(f'{passing_pbits_not_daughter}')
print()

# Jim's solution

https://stackoverflow.com/questions/78657390/accessing-elements-of-an-awkward-array-that-are-not-a-passed-in-index/78659051#78659051

In [ ]:
def indices_to_booleans(indices, array_to_slice):
    whole_set, in_set = ak.unzip(ak.cartesian([
        ak.local_index(array_to_slice), indices
    ], nested=True))
    return ak.any(whole_set == in_set, axis=-1)

In [ ]:
jet_lepton_indices = ak.Array([[0, 2], [1], [2,3]])
print(f'jet_lepton_indices\n{jet_lepton_indices}\n')

lepton_flags = ak.Array([[0, 10, 20, 30], [0, 10, 20, 30], [0, 10, 20, 30, 40]])
print(f'lepton_flags\n{lepton_flags}\n')

In [ ]:
idx = indices_to_booleans(jet_lepton_indices, lepton_flags)

idx

In [ ]:
lepton_flags[idx]

In [ ]:
lepton_flags[~idx]

With our code.

In [ ]:
def build_antiproton_antimask(ak_arr, pps, selector = 'SuperLooseKMProtonSelection', verbose=0):

    if verbose:
        print("The MC and tracks for the first entry")
        idx = ak_arr['TRKMCIdx'][0]
        mclund = ak_arr['mcLund'][0]

        for i,id in enumerate(idx):
            print(f"{i:2d}  {id:4d}   {mclund[id]}")
    print()

    
    lamd1idx = ak_arr['Lambda0d1Idx']
    lamd1Lund = ak_arr['Lambda0d1Lund']

    if verbose:
        print(f'lamd1idx\n{lamd1idx}')
        print(f'lamd1Lund\n{lamd1Lund}')
    
    d2idx = ak_arr['Bd2Idx']
    d2Lund = ak_arr['Bd2Lund']

    if verbose:
        print()
        print(f'B d2idx\n{d2idx}')
        print(f'B d2Lund\n{d2Lund}')
        print()

    qBd2 = (ak_arr['Bd2Lund'])/np.abs(ak_arr['Bd2Lund'])
    qBd2 = qBd2[:,0]

    qlamd1 = (ak_arr['Lambda0d1Lund'])/np.abs(ak_arr['Lambda0d1Lund'])
    qlamd1 = qlamd1[:,0]

    if verbose:
        print("Proton charges ----------------")
        print(f"qBd2\n{qBd2}\n")
        print(f"qlamd1\n{qlamd1}\n")
        print("Proton charges ----------------\n\n")

    qtrk = (ak_arr['TRKLund'])/np.abs(ak_arr['TRKLund'])

    if verbose:
        print("Track charges ----------------")
        print(f"qtrk\n{qtrk}\n")
        print("Track charges ----------------\n\n")
    
    
    trkidx_proton = ak_arr['pTrkIdx']

    if verbose:
        print(f"# of protons: {ak_arr['np']}")
        print(f"trkidx_proton (the track index for labeled protons) \n{trkidx_proton}")
        print()
    
    lamd1_trkidx = trkidx_proton[lamd1idx]
    d2_trkidx = trkidx_proton[d2idx]

    if verbose:
        print(f"lamd1_trkidx\n{lamd1_trkidx}\n")    
        print(f"B d2_trkidx\n{d2_trkidx}\n")
    

    trk_selector_map = ak_arr['pSelectorsMap']
    
    if verbose:
        print(f"qtrk\n{qtrk}\n")
        print(f"proton trk_selector_map\n{trk_selector_map}\n")
        print(f"pion   trk_selector_map\n{ak_arr['piSelectorsMap']}\n")
        print(f"kaon   trk_selector_map\n{ak_arr['KSelectorsMap']}\n")

    lamproton_selector_map = trk_selector_map[lamd1_trkidx]

    if verbose:
        print(lamproton_selector_map)
        print()
    
    bool_idx1 = indices_to_booleans(lamd1_trkidx, trk_selector_map)
    bool_idx2 = indices_to_booleans(d2_trkidx, trk_selector_map)

    if verbose:
        print(f"proton track selectors for the index of the proton from the lambda (and not that)")
        print(f"The boolean mask bool_idx1\n{bool_idx1}")
        print(trk_selector_map[bool_idx1])
        print(trk_selector_map[~bool_idx1])
        print()

        print(f"proton track selectors for the index of the proton from the B (and not that)")
        print(f"The boolean mask bool_idx1\n{bool_idx2}")
        print(trk_selector_map[bool_idx2])
        print(trk_selector_map[~bool_idx2])
        print()
        
    
        # Both protons
        print(f"Boolean for both protons (or) \n{(bool_idx1 | bool_idx2)}")
        print(f"trk selectors for both protons (or) \n{trk_selector_map[bool_idx1 | bool_idx2]}")
        print(f"trk selectors for other protons     \n{trk_selector_map[~(bool_idx1 | bool_idx2)]}")
        print()
    
    
    
    pbits = bat.calculate_bits_for_PID_selector(None, trk_selector_map)

    if verbose:
        print(f"The pbits -----------------")
        print(f'All the pbits\n{pbits}')
        print(f"bool for protons from Lambda\n{bool_idx1}")
        print(f'pbits for protons from Lambda     pbits[bool_idx1]\n{pbits[bool_idx1]}')
        print(f'pbits for protons not from Lambda pbits[~bool_idx1]\n{pbits[~bool_idx1]}')
        print()

        print(f"bool for protons from B\n{bool_idx2}")
        print(f'pbits for protons from B     pbits[bool_idx2]\n{pbits[bool_idx2]}')
        print(f'pbits for protons not from B pbits[~bool_idx2]\n{pbits[~bool_idx2]}')
        print()

        
    mask_bool = bat.mask_PID_selection(pbits, selector, pps)

    if verbose:
        print(f"mask_bool, (mask_PID_selection) the tracks that pass {selector}")
        print(mask_bool)
        print(pbits[mask_bool])
        print(pbits[~mask_bool])
        print()
        
        print(f"pbits that are not the final state lambda but are protons")
        print(pbits[~bool_idx1 & mask_bool])
        print()
        
        print(f"TRKcharge that are not the final state lambda but are protons")
        print(qtrk[~bool_idx1 & mask_bool])
        print()
    
    charge_test_idx1 = qtrk[~bool_idx1 & mask_bool] == -qlamd1
    charge_test_idx2 = qtrk[~bool_idx2 & mask_bool] == -qBd2

    if verbose:
        print(f"charge test idx1\n{charge_test_idx1}")
        print(f"charge test idx2\n{charge_test_idx2}")
    

    # This is True if there is an antiproton in the other final state particles
    # This is likely (I think) with background and less likely with signal
    charge_test = qtrk[~(bool_idx1 | bool_idx2) & mask_bool] == -qlamd1

    if verbose:
        print(f"charge test both: is neither Lam/B dau, passes selectors, opposite charge\n{charge_test}")

    print("\nCounting..............")
    print(f'{charge_test[charge_test] = }')
    print(f'{ak.num(charge_test[charge_test]) = }')
    print(f'{ak.sum(ak.num(charge_test[charge_test])) = }')
    print()
    
    #nhave_opposite = ak.sum(ak.num(charge_test[charge_test]))
    #n = ak.num(charge_test, axis=0)
    
    have_opp = ak.num(charge_test[charge_test])
    
    nhave_opp = len(have_opp[have_opp>0])

    ndont_have_opp = len(have_opp[have_opp==0])

    n = len(ak_arr)

    print(f"# of events:                                  {n}")
    print(f"# of events that don't have opposite protons: {ndont_have_opp}")
    print(f"# of events that       have opposite protons: {nhave_opp}")

    return charge_test


#spmask = ak_sp['spmode']=='-999'
spmask = ak_sp['spmode']=='998'
#spmask = ak_sp['spmode']=='1005'
#spmask = ak_sp['spmode']=='991'

# 998 and 1000 to 1002 show that I am not counting things correctly. 
# Maybe works npe? 8/11/2024

pps = myPIDselector.PIDselector("p")

# Do this with 998
# 1001 to 1002 has an opposite proton
# 1004 to 1004 has no opposite protons
ct = build_antiproton_antimask(ak_sp[spmask][1004:1005], pps, 'TightKMProtonSelection', verbose=1)
#ct = build_antiproton_antimask(ak_sp[spmask], pps, 'LooseKMProtonSelection', verbose=0)

# 1005
#ct = build_antiproton_antimask(ak_sp[spmask][100:101], pps, 'TightKMProtonSelection', verbose=True)
#ct = build_antiproton_antimask(ak_sp[spmask][1000:1001], pps, 'TightKMProtonSelection', verbose=1)
#ct = build_antiproton_antimask(ak_sp[spmask], pps, 'LooseKMProtonSelection', verbose=0)

# When I run this over 998 backgrounds, I get about 1/2 have opposite protons and 1/2 do not
# It looks like I get the no opposite protons because there aren't real protons in the event to start!

# Make a plot for each of the MCs as to what fraction of events contain real protons
print(ct)

In [ ]:
n = 0
print(ak_sp['TRKMCIdx'][n])

print(ak_sp['mcLund'][n])
print()

idx = ak_sp['TRKMCIdx'][n]
mclund = ak_sp['mcLund'][n]

for i,id in enumerate(idx):
    print(f"{i:2d}  {id:4d}   {mclund[id]}")

In [ ]:
x = ak.Array([[True, False], [False, False, False]])

print(x[x])

n = ak.num(x[x])
print(n>0)

print(len(n[n>0]))

In [ ]:
# Fraction of events with proton

#spcut = ak_sp['spmode'] == '1005'
spcut = ak_sp['spmode'] == '998'

mclund = ak_sp[spcut]['mcLund']
idx = ak_sp[spcut]['TRKMCIdx']

trk_mclund = mclund[idx]
print(trk_mclund)

test_for_proton = (trk_mclund == 2212) | (trk_mclund == -2212)

for t in trk_mclund[0:10]:
    print(t)

test = ak.any(test_for_proton, axis=-1)

print(test)

print(len(test))
print(len(test[test]))


In [ ]:
#ak.ones_like(lamd1_trkidx)

In [ ]:
#ak.local_index(trk_selector_map)

In [ ]:
x = ak.Array([[-1, 1], [1, 1, 1, -1]])
y = ak.Array([1, -1])

x==y